# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [108]:
%%time
import pandas as pd
import numpy as np
import datetime


def escrever_csv(dfa, nome): 
    dfa.to_csv('D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\%s.csv' %nome, index=False)

CPU times: total: 0 ns
Wall time: 0 ns


---

#  <span style="color:red"><u>Ler Ficheiro Completo</u> </span>

In [162]:
%%time

#Ler os ficheiros
df_2022 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2022\\Stocks_Delta_2022_Limpo.csv')
df_2023 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2023\\Stocks_Delta_2023_Limpo.csv')

# Juntar as bases
dataframes = [df_2022, df_2023]
df_Fusão = pd.concat(dataframes, ignore_index=True)
df_Fusão['DATA']= pd.to_datetime(df_Fusão['DATA'], format='%Y-%m-%d')

# Ficheiro de previsão
df_Prophet = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\Prophet.csv')
df_Prophet['DATA']= pd.to_datetime(df_Prophet['DATA'], format='%Y-%m-%d')

df_XGBoost = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\XGBoost.csv')
df_XGBoost['DATA']= pd.to_datetime(df_XGBoost['DATA'], format='%Y-%m-%d')

CPU times: total: 22.5 s
Wall time: 30.6 s


In [166]:
print(len(df_2023.DESC_ARTIGO.unique()))
print(len(df_2023.STORE.unique()))

181
334


# <font color=red>Opcional:</font> Definir produtos em causa

- Produtos específicos

In [110]:
%%time
# Ler ficheiro dos produtos e lojas para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\lojas.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()
produtos+=["CAFÉ DELTA Q MYTHIQ 10CAP", "CAFÉ DELTA Q MYTHIQ XL 40CAP", "CAFÉ DELTA Q QALIDUS 10CAP", "CAFÉ DELTA Q QALIDUS 40CAP",
                'CAFÉ DELTA Q QHARACTER 10CAP','CAFÉ DELTA Q QHARACTER 40CAP','CAFÉ DELTA Q DEQAFEINATUS 10CAP','CAFÉ DELTA Q DEQAFEINATUS XL 40CAP',
               'CAFÉ DELTA MOAGEM UNIVERSAL ANGOLA 220G','CAFÉ DELTA MOAGEM UNIVERSAL BRASIL 220G']
# Alterar o dataframe para apenas incluir os produtos e lojas em causa
dfFinal = df_Fusão[(df_Fusão["DESC_ARTIGO"].isin(produtos)) & (df_Fusão["STORE_NAME"].isin(lojas))].copy()

CPU times: total: 1.77 s
Wall time: 1.79 s


## <font color=red>Fim</font> 

In [111]:
#dfFinal = df_Fusão[(df_Fusão["DESC_ARTIGO"].isin(produtos))].copy()

In [112]:
dfFinal['SELLOUT'] = dfFinal.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfFinal['STOCK'] = dfFinal.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)

In [113]:
dfFinal.head()

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,SELLOUT,SELLOUT_1_Dias_Antes
60608,2022-01-01,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,151.0,151.0,0.0,11.0
60609,2022-01-02,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,137.0,151.0,14.0,0.0
60610,2022-01-03,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,48,0,120,175.0,137.0,10.0,14.0
60611,2022-01-04,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,162.0,175.0,13.0,10.0
60612,2022-01-05,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,195.0,162.0,15.0,13.0


- Fundir o Ficheiro da Delta com a previsão

In [114]:
%%time
# Prophet
dfFinal = pd.merge(dfFinal, df_Prophet[['DATA', 'STORE', 'DESC_ARTIGO', 'Prophet']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )
# XGBoost
dfFinal = pd.merge(dfFinal, df_XGBoost[['DATA', 'STORE', 'DESC_ARTIGO', 'XGBoost']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )

CPU times: total: 594 ms
Wall time: 634 ms


# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA

In [115]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

> - PRÉ_ROTURA

In [116]:
# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)

# Colunas de métricas 30, 60, 120 e 180 dias antes

> - Volatilidade de Procura: <br>
coeficiente de variação

In [117]:
%%time
dfFinal["Volatilidade_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))

dfFinal["Volatilidade_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))

dfFinal["Volatilidade_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).std())/
                              dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))

dfFinal["Volatilidade_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).std())/
                              dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))



CPU times: total: 125 ms
Wall time: 122 ms


> - Percentagem de Rotura: <br>
média de roturas $* 100$

In [118]:
%%time
#historico("Percentagem_Roturas", "ROTURA", pd.Series.mean)

dfFinal["Percentagem_Roturas_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


CPU times: total: 62.5 ms
Wall time: 62.3 ms


> - Percentagem de Supply:<br>
média de vezes que foi pedido stock $*100$

In [119]:
dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED"].shift(1)==0) & (dfFinal["EXPECTED"]>0), 1, 0)

In [120]:
#historico("Percentagem_Supply", "New_Supply", pd.Series.mean)

dfFinal["Percentagem_Supply_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum())/30)*100

dfFinal["Percentagem_Supply_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum())/60)*100

dfFinal["Percentagem_Supply_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum())/120)*100

dfFinal["Percentagem_Supply_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum())/180)*100


> - Efeito fim de semana

In [121]:
dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()

In [122]:
dfFinal["Efeito_Fds_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())))-1

dfFinal["Efeito_Fds_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())))-1

dfFinal["Efeito_Fds_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())))-1

dfFinal["Efeito_Fds_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())))-1

In [123]:
dfFinal.head(15)

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,...,Percentagem_Supply_30,Percentagem_Supply_60,Percentagem_Supply_120,Percentagem_Supply_180,SELLOUT_fds,SELLOUT_semana,Efeito_Fds_30,Efeito_Fds_60,Efeito_Fds_120,Efeito_Fds_180
0,2022-01-01,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,151.0,151.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,137.0,151.0,...,0.000000,0.000000,0.000000,0.000000,14.0,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,48,0,120,175.0,137.0,...,0.000000,0.000000,0.000000,0.000000,NaN,10.0,NaN,NaN,NaN,NaN
3,2022-01-04,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,162.0,175.0,...,0.000000,0.000000,0.000000,0.000000,NaN,13.0,-0.300000,-0.300000,-0.300000,-0.300000
4,2022-01-05,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,195.0,162.0,...,0.000000,0.000000,0.000000,0.000000,NaN,15.0,-0.391304,-0.391304,-0.391304,-0.391304
5,2022-01-06,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,184.0,195.0,...,3.333333,1.666667,0.833333,0.555556,NaN,11.0,-0.461538,-0.461538,-0.461538,-0.461538
6,2022-01-07,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,174.0,184.0,...,3.333333,1.666667,0.833333,0.555556,10.0,NaN,-0.416667,-0.416667,-0.416667,-0.416667
7,2022-01-08,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,137.0,174.0,...,3.333333,1.666667,0.833333,0.555556,37.0,NaN,-0.166667,-0.166667,-0.166667,-0.166667
8,2022-01-09,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,118.0,137.0,...,3.333333,1.666667,0.833333,0.555556,19.0,NaN,0.000000,0.000000,0.000000,0.000000
9,2022-01-10,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,109.0,118.0,...,3.333333,1.666667,0.833333,0.555556,NaN,9.0,0.166667,0.166667,0.166667,0.166667


> - Tempo médio inter-supply

In [124]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [125]:
#historico("InterSupplyMed", "InterSupply", pd.Series.mean)

dfFinal["InterSupplyMed_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Tempo indisponível

In [126]:
'''O que vai acontecer é: no primeiro dia em que há rotura vai aparecer a soma de todos os dias com rotura a seguir a esse!
Todos os outros valores serão NaN para não serem considerados quando for feita a média. Assim a média corresponderá ao
número médio de dias em que se deixa um produto em rotura.'''

dfFinal["Tempo_Indisponível"] = np.where(dfFinal["ROTURA"]==1, 1, 0)

groups = (dfFinal['Tempo_Indisponível'] != dfFinal['Tempo_Indisponível'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'Tempo_Indisponível': 'sum'}).reset_index(drop=True)
result = result[result['Tempo_Indisponível'] > 0]

dfFinal = dfFinal.drop(columns=['Tempo_Indisponível'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [127]:
#historico("Percentagem_Supply", "New_Supply", pd.Series.mean)

dfFinal["Tempo_Indisponível_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))

dfFinal["Tempo_Indisponível_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))

dfFinal["Tempo_Indisponível_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))

dfFinal["Tempo_Indisponível_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))


> - Percentagem de dias em Stock Borderline

In [128]:
dfFinal["Stock_Borderline"] = np.where(dfFinal["STOCK"]<0.2*dfFinal["PRES_STOCK"], 1, 0)

In [129]:
dfFinal["Tempo_Stock_Borderline_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias de Linear Incompleto

In [130]:
dfFinal["Linear_Incompleto"] = np.where(dfFinal["STOCK"]<dfFinal["PRES_STOCK"], 1, 0)

In [131]:
dfFinal["Tempo_Linear_Incompleto_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias sem vendas

In [132]:
dfFinal["Sem_Vendas"] = np.where(dfFinal["SELLOUT"] == 0, 1, 0)

In [133]:
#historico("Sem_Vendas", "Sem_Vendas", pd.Series.mean)

dfFinal["Sem_Vendas_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Vendas perdidas

In [134]:
dfFinal['ROTURA_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["ROTURA"].copy()
dfFinal['ROTURA_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["ROTURA"].copy()

In [135]:
'''Mediana de fins de semana e mediana de semana a multiplicar pelo nº de dias em que há rotura, soma dos valores para
ter as perdas de vendas estimadas'''

#mediana fds*roturas fds + mediana semana*roturas semana

dfFinal["Vendas_Perdidas_30"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())) + 
                                (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_60"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())) + 
                                (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_120"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())) + 
                                 (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_180"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())) + 
                                 (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median()))
)

# Métricas 4, 5 e 10 antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [136]:
# Quantos dias antes:

diasMet = [4, 5, 10]

> Função

In [137]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [138]:
%%time
# Usar função para sellouts até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 125 ms
Wall time: 128 ms


> - STOCKS

In [139]:
# Usar função para Stocks até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "STOCK")

> > - Ordenar

> - INTRANSIT e EXPECTED

In [140]:
# Usar função para Trânsito até 10 dias antes


for i in diasMet:
    dias(dfFinal, i, "INTRANSIT")
    
for i in diasMet:
    dias(dfFinal, i, "EXPECTED")

> - STK

In [141]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in diasMet:
    dias(dfFinal, i, "STK")

> - MSA

In [142]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [143]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]

for i in diasMet:
    dias(dfFinal, i, "CICLOS")

> - Dias para rotura de Stock

In [144]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STOCK"] / dfFinal["MSA10"]

for i in diasMet:
    dias(dfFinal, i, "Dias_para_Rotura_Stock")

> - Dias para rotura de Linear

In [145]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duração_Linear'] = dfFinal["PRES_STOCK"] / dfFinal["MSA10"]

for i in diasMet:
    dias(dfFinal, i, "Dias_Duração_Linear")

> - Adequação de Stock

In [146]:
# Coluna de adequação de stock


dfFinal["Adequação"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in diasMet:
    dias(dfFinal, i, "Adequação")

>- Balance: sellout / soma stock disponível mais transito.

In [147]:
# Colunas de balanço


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in diasMet:
    
    
    valores = (i+1) * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


>- Balance optimizado

In [148]:


    
dfFinal["Balance_Optimized"] = np.where((dfFinal["MSA10Dp"] / dfFinal["MSA10"]) * 100 > 100, dfFinal["MSA20"] / dfFinal["STK"],
                                dfFinal["MSA10"] / dfFinal["STK"])   

for i in diasMet:
    
    
    valores = (i+1) * dfFinal["Balance_Optimized"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Optimized", i)] = valores

> - Mediano é com o mínimo dos ultimos 10 dias

In [149]:
# Coluna de adequação de stock


# MSA_med do dia = mediana dos sellouts dos 10 dias anteriores (exclui o próprio dia)
# dfFinal["MdSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Mediano"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median()) / dfFinal["STK"]

for i in diasMet:
    
    
    valores = (i+1) * dfFinal["Balance_Mediano"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Mediano", i)] = valores



> - Liberal é com o mínimo dos ultimos 10 dias

In [150]:
# Liberal


# MSA do dia = média dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["LSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Liberal"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min()) / dfFinal["STK"]

for i in diasMet:
    
    
    valores = (i+1) * dfFinal["Balance_Liberal"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Liberal", i)] = valores



> - Conservador é com o máximo dos ultimos 10 dias

In [151]:
# Conservador

# MSA_max do dia = máximo dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["CSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max())
 
    
# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Conservador"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max()) / dfFinal["STK"]

for i in diasMet:
    
    # Multiplicar os balances pelo número de dias+1 antes do dia actual
    valores = (i+1) * dfFinal["Balance_Conservador"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Conservador", i)] = valores



> - Smart é com semana e fim de semana

In [152]:
%%time
import pandas as pd
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def calculate_weekday_weekend_counts(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(calculate_weekday_weekend_counts)




CPU times: total: 25.5 s
Wall time: 25.6 s


In [153]:
# Smart



dfFinal["Balance_Smart_5_Dias_Antes"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].transform(lambda x: x.rolling(window=30, min_periods=1).median())
     * dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].transform(lambda x: x.rolling(window=30, min_periods=1).median())
     * dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK"])

In [154]:
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfFinal = pd.merge(dfFinal, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

# Escrever

- 0's e 1's

- Dias certos

In [155]:
dfEscrever2022 = dfFinal.copy()
dfEscrever2023 = dfFinal.loc[(dfFinal['DATA'] >= '2023-01-01') ].copy()

#df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
#df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

- Passar para csv

In [161]:
dfFinal.STORE_NAME.unique()

array(['CNT MATOSINHOS', 'CNT AMADORA', 'CNT CASCAIS', 'CNT LEIRIA',
       'CNT COIMBRASHOPPING', 'CNT PORTIMÃO', 'CNT VIANA', 'CNT SANTAREM',
       'MDL T. VEDRAS', 'CNT BRAGA'], dtype=object)

In [156]:
%%time

escrever_csv(dfEscrever2022, "FinalDataBaseDeltaTotal")
escrever_csv(dfEscrever2023, "FinalDataBaseDeltaTotal2023")

CPU times: total: 11.3 s
Wall time: 13.3 s


In [10]:
dfNinjas2 = pd.read_csv("D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\CHMetNaoSmart2.csv")

In [15]:
print(dfNinjas2.shape)
print(dfNinjas.shape)

(24290, 129)
(24290, 130)


In [23]:
dfNinjas2.columns.tolist()

['DATA',
 'EAN',
 'DESC_ARTIGO',
 'STORE',
 'STORE_NAME',
 'INTRANSIT',
 'EXPECTED',
 'PRES_STOCK',
 'STOCK',
 'STOCK_1_Dias_Antes',
 'SELLOUT',
 'SELLOUT_1_Dias_Antes',
 'Prophet',
 'XGBoost',
 'ROTURA',
 'PRE_ROTURA',
 'Volatilidade_30',
 'Volatilidade_60',
 'Volatilidade_120',
 'Volatilidade_180',
 'Percentagem_Roturas_30',
 'Percentagem_Roturas_60',
 'Percentagem_Roturas_120',
 'Percentagem_Roturas_180',
 'New_Supply',
 'Percentagem_Supply_30',
 'Percentagem_Supply_60',
 'Percentagem_Supply_120',
 'Percentagem_Supply_180',
 'SELLOUT_fds',
 'SELLOUT_semana',
 'Efeito_Fds_30',
 'Efeito_Fds_60',
 'Efeito_Fds_120',
 'Efeito_Fds_180',
 'InterSupply',
 'InterSupplyMed_30',
 'InterSupplyMed_60',
 'InterSupplyMed_120',
 'InterSupplyMed_180',
 'Tempo_Indisponível',
 'Tempo_Indisponível_30',
 'Tempo_Indisponível_60',
 'Tempo_Indisponível_120',
 'Tempo_Indisponível_180',
 'Stock_Borderline',
 'Tempo_Stock_Borderline_30',
 'Tempo_Stock_Borderline_60',
 'Tempo_Stock_Borderline_120',
 'Tempo_Sto

In [2]:
# Ler ficheiro
dfNinjas = pd.read_csv("D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\CHMetNaoSmart.csv")


ValueError: Worksheet named 'Lista Lojas Sonae' not found

In [14]:
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

In [55]:
escrever_csv(dfFinal2, "Mil_MétricasCerto07")
#escrever_csv(df_DiasCertos, "StocksDelta_2023_10prod_Limpo")
#escrever_csv(df_RoturasDiasCertos, "StocksDelta_2023_10prod_Roturas")

In [4]:
def escrever_excel(dataFrame, nomeFicheiro):
    dataFrame.to_excel('D:\\B&N Dados\\Delta\\Piloto\\%s.xlsx' %nomeFicheiro, index=False)
    

In [5]:
escrever_excel(oi, "ParaTestarCoisas")

In [4]:
oiEscrever = oi[("2023-04-14"<=oi["DATA"]<="2023-04-16") or ("2023-04-21"<=oi["DATA"]<="2023-04-23") or ("2023-04-28"<=oi["DATA"]<="2023-04-30") or ("2023-05-05"<=oi["DATA"]<="2023-05-07")]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [87]:
filtered_oi = oi[oi["DATA"].between("2023-04-14", "2023-04-16") |
                  oi["DATA"].between("2023-04-21", "2023-04-23") |
                  oi["DATA"].between("2023-04-28", "2023-04-30") |
                  oi["DATA"].between("2023-05-05", "2023-05-07")]

In [19]:
dfNinjas["Vendedor"]

KeyError: 'Vendedor'

# Clickhouse

In [169]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

In [175]:
#Estabelecer a base a ser lida

baseCH = dfEscrever2023.copy()
#2023
tabela = "FinalDataBaseDelta2023"

In [176]:

client = clickhouse_connect.get_client(host='e28fluocjc.europe-west4.gcp.clickhouse.cloud', 
                                       port=8443, 
                                       username='default', 
                                       password='eKn4CWkTDFpi_',
                                       database='Delta')



###
baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar para datetime
new_columns = [unidecode(col) for col in baseCH.columns]           # Tirar acentos e afins
baseCH.columns = new_columns                                       # Aplicar alterações da linha anterior
###

# Tipos de dados
data = ["DATA"]
texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


# Só floats é que permitem missing values
for col_name in inteiros:
    baseCH[col_name] = baseCH[col_name].astype(float)
# Missing values em strings estragam tudo
baseCH[texto] = baseCH[texto].fillna("-")


def schema(lista, tipo):
    result_list = [f"{element} {tipo}" for element in lista]
    return result_list

data1 = schema(data, "Date")
texto1 = schema(texto, "String")
inteiros1 = schema(inteiros, "Float64")
floats1 = schema(floats, "Float64")
total = tuple(data1 + texto1 + inteiros1 + floats1)

schema = ', '.join([column.replace("'", "") for column in total])

# Split the input string by commas
parts = schema.split(', ')
# Process each part and wrap the first word in double quotes
output_parts = []
for part in parts:
    words = part.split()
    if words:
        first_word = words[0]
        remaining_words = ' '.join(words[1:])
        output_part = f'"{first_word}" {remaining_words}'
        output_parts.append(output_part)
# Join the modified parts back into a string
schema = ', '.join(output_parts)



# Eliminar tabela no CH
client.command(f'DROP TABLE IF EXISTS {tabela}')

# Criar tabela no CH
client.command(f'''
    CREATE TABLE IF NOT EXISTS {tabela} (
        {schema}
        ) ENGINE = MergeTree
        ORDER BY (DATA)
''')

client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())